In [97]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
#local imports

sys.path.append(os.path.abspath(os.path.join('..')))
from src.data.manipulate import Manipulate


In [47]:
#read file
train_df = pd.read_csv('../data/raw/train.csv', dtype={'StateHoliday': object})
test_df = pd.read_csv('../data/raw/test.csv')
store_df = pd.read_csv('../data/raw/store.csv')

In [48]:
train_df = pd.merge(train_df, store_df, how= 'left', on='Store')
test_df = pd.merge(test_df, store_df, how= 'left', on='Store')

In [49]:
#convert the tabular data to a time series data
ID = test_df['Id']
test_df.drop('Id', inplace=True, axis=1)
train_df.sort_values(['Store', 'Date'], ignore_index=True, inplace=True)
test_df.sort_values(['Store', 'Date'], ignore_index=True, inplace=True)

for dataset in (train_df, test_df):
    dataset['Date'] = pd.to_datetime(dataset['Date'])
    dataset['Day'] = dataset.Date.dt.day
    dataset['Month'] = dataset.Date.dt.month
    dataset['Year'] = dataset.Date.dt.year
    dataset['DayOfYear'] = dataset.Date.dt.dayofyear
    dataset['WeekOfYear'] = dataset.Date.dt.weekofyear
    dataset.set_index('Date', inplace=True)

In [50]:
test_df.tail()

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Day,Month,Year,DayOfYear,WeekOfYear
Date,,,,,,,,,,,,,,,,,,,,
2015-09-13,1115,7,0.0,0,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",13,9,2015,256,37
2015-09-14,1115,1,1.0,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",14,9,2015,257,38
2015-09-15,1115,2,1.0,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",15,9,2015,258,38
2015-09-16,1115,3,1.0,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",16,9,2015,259,38
2015-09-17,1115,4,1.0,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",17,9,2015,260,38


In [98]:
manipulate = Manipulate(train_df)
# Fill missing numeric values
manipulate.fill_columns_with_max(train_df.select_dtypes(exclude=['object']).columns.tolist())
# Fill non-numeric values (categorical values)
manipulate.fill_columns_with_most_frequent(train_df.select_dtypes(include=['object']).columns.tolist())


In [100]:
manipulate.df.isna().any()

Store                        False
DayOfWeek                    False
Weekend                      False
WeekDay                      False
Sales                        False
Customers                    False
Open                         False
Promo                        False
StateHoliday                 False
SchoolHoliday                False
StoreType                    False
Assortment                   False
CompetitionDistance          False
CompetitionOpenSinceMonth    False
CompetitionOpenSinceYear     False
Promo2                       False
Promo2SinceWeek              False
Promo2SinceYear              False
PromoInterval                False
Day                          False
Month                        False
Year                         False
DayOfYear                    False
WeekOfYear                   False
dtype: bool

### Extract the following from the datetime column

1. **Weekdays**

In [51]:
#set the weekday 1 for DayOfWeek b/n 1 and 5 and 0 for 6 and 7
day_of_week_index = train_df.columns.get_loc('DayOfWeek')
train_df.insert(day_of_week_index + 1, 'WeekDay', train_df['DayOfWeek'].apply(lambda x: 1 if x <= 5 else 0) )


2. **Weekends**

In [64]:
#set the weekend 1 for DayOfWeek equals 6 & 7 and 0 for the rest
day_of_week_index = train_df.columns.get_loc('DayOfWeek')
train_df.insert(day_of_week_index + 1, 'Weekend' ,train_df['DayOfWeek'].apply(lambda x : 0 if x <= 5 else 1))
train_df.head(100)

1

3. **Number of days to holidays**

In [101]:
manipulate.add_number_of_days_to_holiday('StateHoliday')

Data Manipulatior:INFO->Successfully Added DaysToHoliday Column


4. **Number of days after holidays**

In [102]:
manipulate.add_number_of_days_after_holiday('StateHoliday')

Data Manipulatior:INFO->Successfully Added DaysAfterHoliday Column


5. **Beginning of month, mid month, and ending of month**

In [103]:
manipulate.add_month_timing('Day')

Data Manipulatior:INFO->Successfully Added MonthTiming Column


In [105]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 27 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Weekend                    1017209 non-null  int64  
 3   WeekDay                    1017209 non-null  int64  
 4   Sales                      1017209 non-null  int64  
 5   Customers                  1017209 non-null  int64  
 6   Open                       1017209 non-null  int64  
 7   Promo                      1017209 non-null  int64  
 8   StateHoliday               1017209 non-null  object 
 9   DaysAfterHoliday           1017209 non-null  int64  
 10  DaysToHoliday              1017209 non-null  int64  
 11  SchoolHoliday              1017209 non-null  int64  
 12  StoreType                  1017209 non-null  object 
 1

### Drop Unimportant columns 

In [108]:
train_df.drop(['Store','Customers'],axis=1,inplace=True)

In [109]:
train_df.to_csv('../data/processed/unlabeled_train_data.csv')

### Encode Categorical Columns

In [112]:
# Label Object type data columns (StoreType,Assortment,StateHoliday,Season)

manipulate.label_columns(['Season', 'StateHoliday', 'StoreType', 'Assortment'])


Failed to Label Encode columns


In [113]:
train_df['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     801153
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: PromoInterval, dtype: int64

In [114]:
manipulate.label_columns(['PromoInterval'])

{'PromoInterval': LabelEncoder()}

In [115]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 25 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  int64  
 1   Weekend                    1017209 non-null  int64  
 2   WeekDay                    1017209 non-null  int64  
 3   Sales                      1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   DaysAfterHoliday           1017209 non-null  int64  
 8   DaysToHoliday              1017209 non-null  int64  
 9   SchoolHoliday              1017209 non-null  int64  
 10  StoreType                  1017209 non-null  object 
 11  Assortment                 1017209 non-null  object 
 12  CompetitionDistance        1017209 non-null  float64
 1

### Data Scaling

In [117]:
def get_min_max_of_dataframe_columns(df):
    top = df.max()
    top_df = pd.DataFrame(top, columns=['Max Value'])
    bottom = df.min()
    bottom_df = pd.DataFrame(bottom, columns=['Min Value'])
    info_df = pd.concat([top_df, bottom_df], axis=1)
    return info_df

In [118]:
# Using StandardScaler to standardize the all columns
scale_list = train_df.columns.to_list()
# Scale
manipulate.standardize_columns(scale_list)
get_min_max_of_dataframe_columns(train_df)

Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column
Failed to standardize the column


,Max Value,Min Value
DayOfWeek,1.502791,-1.501129
Weekend,1.585611,-0.630672
WeekDay,0.630672,-1.585611
Sales,9.292957,-1.499723
Open,0.452399,-2.21044
Promo,1.273237,-0.7854
StateHoliday,c,0
DaysAfterHoliday,18,0
DaysToHoliday,135,0
SchoolHoliday,1,0


### Save Clean Data

In [119]:
train_df.to_csv('../data/processed/train.csv')